# Indice espectrales NBR Analisis en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio SHP

In [3]:
import os

In [4]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP\Indices"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\Indices'

In [5]:
import geopandas as gpd

In [6]:
# cargar un shp
zona = gpd.read_file("Zona_Quema_Geo.shp")
zona

,id,geometry
0,1,"POLYGON ((-118.46472 34.47016, -118.24337 34.4..."


In [7]:
# Crear un featureCollecion mediante GeoJSON
ee_zona = ee.FeatureCollection(zona.__geo_interface__)

In [8]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [9]:
geometria = ee_zona.geometry()
Map.centerObject(geometria, 11)

In [10]:
# Visualizar 
Map.addLayer(ee_zona, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[34.390248284782864, -118.35251181929958], controls=(WidgetControl(options=['position', 'transparen…

## Seleccion de imagenes

In [11]:
# Seleccionar una imagen Landsat8
img_L8_antes = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_041036_20160708")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32711", scale = 30)\
                  .clip(ee_zona)

In [12]:
# Seleccionar una imagen Landsat8
img_L8_despues = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_041036_20160724")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32711", scale = 30)\
                  .clip(ee_zona)

## Consultar Nombre bandas y proyección

In [13]:
# Nombre de las bandas
print("Nombre bandas L8 Antes: ",img_L8_antes.bandNames().getInfo())
print("Nombre bandas L8 Despues: ",img_L8_despues.bandNames().getInfo())

Nombre bandas L8 Antes:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas L8 Despues:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [14]:
# Consultar Proyeccion
print("Proyeccion L8 Antes: ",img_L8_antes.projection().getInfo())
print("Proyeccion L8 Despues: ",img_L8_despues.projection().getInfo())

Proyeccion L8 Antes:  {'type': 'Projection', 'crs': 'EPSG:32711', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion L8 Despues:  {'type': 'Projection', 'crs': 'EPSG:32711', 'transform': [30, 0, 0, 0, -30, 0]}


In [15]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [16]:
# Visualizar del mapa
Map.addLayer(img_L8_antes, viz_es, name = "Landsat8 Antes")
Map.addLayer(img_L8_despues, viz_es, name = "Landsat 8 Despues")
Map

Map(bottom=209049.0, center=[34.390248284782864, -118.35251181929958], controls=(WidgetControl(options=['posit…

## Determinar NBR

In [17]:
# Funcion para NBR
def Calc_NBR(image):
    nbr = image.expression("float(nir - swir2)/(nir + swir2)", {
        'nir': image.select('NIR'),
        'swir2': image.select('SWIR2')
    })
    return nbr.rename("NBR")

In [21]:
# Calcular indices con funcion
nbr_L8_antes = Calc_NBR(img_L8_antes)
nbr_L8_despues = Calc_NBR(img_L8_despues)

In [19]:
# Simbologia de NBR
viz_nbr = {
    'palette': ["#66FF66","#FF99FF","#66FFFF","#FFFF99",
                "#FF9966","#FF6600","#FF0000"],
    'min': -0.4,
    'max': 0.8,
    'bands': "NBR"
}

In [23]:
# Visualizar 
Map.addLayer(nbr_L8_antes, viz_nbr, name = "NBR antes")
Map.addLayer(nbr_L8_despues, viz_nbr, name = "NBR despues")
Map

Map(bottom=209048.0, center=[34.39104576945997, -118.27434539794923], controls=(WidgetControl(options=['positi…

## Determinar niveles de gravedad

In [24]:
# Determinacion niveles de gravedad dNBR
dNBR = nbr_L8_antes.subtract(nbr_L8_despues)

In [30]:
# clasificacion de NBR 
clas_dnbr = ee.Image(0).where(dNBR.lt(-0.25),1)\
                      .where(dNBR.gte(-0.25),2)\
                      .where(dNBR.gte(-0.1),3)\
                      .where(dNBR.gte(0.10),4)\
                      .where(dNBR.gte(0.27),5)\
                      .where(dNBR.gte(0.44),6)\
                      .where(dNBR.gte(0.66),7)\
                      .clip(ee_zona)\
                      .rename("dNBR")

In [31]:
# Nombre de la banda clasificado
clas_dnbr.bandNames().getInfo()

['dNBR']

In [27]:
# Renombrar el nombre banda
# clas_dnbr = clas_dnbr.rename("dNBR")

In [32]:
# Simbologia de Clase NBR
viz_nbr_clas = {
    'palette': ["#66FF66","#FF99FF","#66FFFF","#FFFF99",
                "#FF9966","#FF6600","#FF0000"],
    'min': 1,
    'max': 7,
    'bands': "dNBR"
}

In [33]:
# Visualizar 
Map.addLayer(clas_dnbr, viz_nbr_clas, name = "NBR clase")
Map

Map(bottom=209057.0, center=[34.38594602425616, -118.28052520751955], controls=(WidgetControl(options=['positi…

## Exportar Imagen

In [34]:
import os
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\Indices'

In [35]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [36]:
# Exportar imagen directorio
geemap.ee_export_image(clas_dnbr, filename="clas_dnbr.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\clas_dnbr.tif


In [37]:
# Exportar en google drive
geemap.ee_export_image_to_drive(clas_dnbr, description='L8_clas_dnbr', folder='GEE_Python', region=geometria, scale=30)

Exporting L8_clas_dnbr ...
